In [2]:
import tensorflow as tf
import numpy as np
import os 
import cv2
import random
from tqdm import tqdm
from tensorflow.keras.layers import LSTM, Bidirectional, Conv2D, Dense, Flatten, MaxPooling2D, TimeDistributed, MaxPoolingD, Reshape
from tensorflow.keras.models import Sequential

In [29]:
def load_data(folder):

    data = []
    for directory in os.listdir(folder):
        if directory == 'fight':
            label = np.array([1, 0])
        elif directory == 'nofight':
            label = np.array([0, 1])
        for files in tqdm(os.listdir(os.path.join(folder, directory))):
            path = os.path.join(os.path.join(folder, directory), files)
            img = cv2.resize(cv2.imread(path), (50, 50))
            data.append(np.array([img, label]))

    return data

In [30]:
train_data = load_data('./Peliculas/Data/train')

100%|██████████| 4410/4410 [00:17<00:00, 250.48it/s]


In [31]:
test_data = load_data('./Peliculas/Data/valid')

100%|██████████| 539/539 [00:01<00:00, 309.39it/s]


In [79]:
cnn = Sequential()
cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(50, 50, 3), padding="same"))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2,2)))
cnn.add(Flatten())
cnn.add(Reshape((-1,  1024)))



lstm_fw = LSTM(units=32)
lstm_bw = LSTM(units=32, go_backwards = True)
model = Sequential()
model.add(TimeDistributed(cnn))
model.add(Bidirectional(lstm_fw, backward_layer = lstm_bw, input_shape=(None, 1, 1024)))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(2, activation='sigmoid'))

cnn.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

cnn.summary()


Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 50, 50, 64)        1792      
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 22, 22, 64)        36928     
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 9, 9, 64)          36928     
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 4, 4, 64)        

In [80]:
random.shuffle(train_data)
random.shuffle(test_data)

In [71]:
X = np.array([i[0] for i in train_data]).reshape(-1, 50, 50, 3)
Y = np.array([i[1] for i in train_data]).reshape(-1, 1, 2)

x_valid = np.array([i[0] for i in test_data]).reshape(-1, 50, 50, 3)
y_valid = np.array([i[1] for i in test_data]).reshape(-1, 1, 2)

In [72]:
X = X / 255.
x_valid = x_valid / 255.

In [73]:
Y.shape

(8605, 1, 2)